In [1]:
"""
Difference in descriptive statistics between train and test							
							
	Delta minimum	Delta maximum	Delta mean	Delta standard deviation	Delta median	Kl divergence	KS test
Column 1							
Column 2							
Column 3							
Column 4							
Column 5							
Column 6							
Column 7							

Load 
1. load imputed_train
   load imputed_test
2. calc min, max, sttdev, mean for train : get_descriptive_statistics_for_columns
 calc min, max, sttdev, mean for test : get_descriptive_statistics_for_columns
3.join dfs :
join select(*(col(x).alias(x + '_pre') for x in preprocessing_cols_stats_df.columns))
joined_df = preprocessing_cols_stats_df_re.join(imputed_cols_stats_df, preprocessing_cols_stats_df_re.column_pre == imputed_cols_stats_df.column)

4. diffrence delts between columns : get_delta_columns_df(joined_df):
5. Caclulate KS divergence and kl divergence
6. K test
7. join delta_df, k_test, kl_divergence
"""


"\nDifference in descriptive statistics between train and test\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\tDelta minimum\tDelta maximum\tDelta mean\tDelta standard deviation\tDelta median\tKl divergence\tKS test\nColumn 1\t\t\t\t\t\t\t\nColumn 2\t\t\t\t\t\t\t\nColumn 3\t\t\t\t\t\t\t\nColumn 4\t\t\t\t\t\t\t\nColumn 5\t\t\t\t\t\t\t\nColumn 6\t\t\t\t\t\t\t\nColumn 7\t\t\t\t\t\t\t\n\nLoad \n1. load imputed_train\n   load imputed_test\n2. calc min, max, sttdev, mean for train : get_descriptive_statistics_for_columns\n calc min, max, sttdev, mean for test : get_descriptive_statistics_for_columns\n3.join dfs :\njoin select(*(col(x).alias(x + '_pre') for x in preprocessing_cols_stats_df.columns))\njoined_df = preprocessing_cols_stats_df_re.join(imputed_cols_stats_df, preprocessing_cols_stats_df_re.column_pre == imputed_cols_stats_df.column)\n\n4. diffrence delts between columns : get_delta_columns_df(joined_df):\n5. Caclulate KS divergence and kl divergence\n6. K test\n7. join delta_df, k_test, kl_diverg

In [2]:

import sys
sys.path.append("/home/boldyrek/mysoft/te/te_reporting/")
from helper_functions import get_imputed_df, start_spark_session, load_df
from col_stats import *
import config as cfg
from helper_functions import *
from scipy import stats

In [3]:
spark = start_spark_session()

In [4]:
test_df = spark.createDataFrame([(1,1,0),(1,1,1),(1,1,1),
                                        (2,1,1),(2,2,0),
                                            (3,1,1),(3,2,0),
                                         (3,3,1),(3,3,0),
                                        (4,3,0),(4,3,0)
                                                  ], ['party_id', 'ctu', 'te_2month'])


In [5]:
# step 1

#imputed_train = load_df( cfg.IMPUTATION_TRAIN_PATH )
#imputed_predict = load_df( cfg.IMPUTATION_PREDICT_PATH )
imputed_train = test_df
imputed_predict = test_df

In [6]:
# step 2
#imputed_train = drop_garbage_cols( imputed_train )
imputed_train_descriptive_stats = get_df_with_descriptive_stats_for_columns ( spark,  imputed_train )
imputed_predict_descriptive_stats = get_df_with_descriptive_stats_for_columns ( spark, imputed_predict )

party_id
ctu
te_2month
[('party_id', 4, 1, 2.45, 1.13, 2.0), ('ctu', 3, 1, 1.91, 0.94, 1.0), ('te_2month', 1, 0, 0.45, 0.52, 0.0)]
party_id
ctu
te_2month
[('party_id', 4, 1, 2.45, 1.13, 2.0), ('ctu', 3, 1, 1.91, 0.94, 1.0), ('te_2month', 1, 0, 0.45, 0.52, 0.0)]


In [7]:
imputed_train_descriptive_stats.show()

+-----------+---+---+----+------+------+
|column_name|min|max|mean|stddev|median|
+-----------+---+---+----+------+------+
|   party_id|  4|  1|2.45|  1.13|   2.0|
|        ctu|  3|  1|1.91|  0.94|   1.0|
|  te_2month|  1|  0|0.45|  0.52|   0.0|
+-----------+---+---+----+------+------+



In [8]:
imputed_predict_descriptive_stats.show()

+-----------+---+---+----+------+------+
|column_name|min|max|mean|stddev|median|
+-----------+---+---+----+------+------+
|   party_id|  4|  1|2.45|  1.13|   2.0|
|        ctu|  3|  1|1.91|  0.94|   1.0|
|  te_2month|  1|  0|0.45|  0.52|   0.0|
+-----------+---+---+----+------+------+



In [9]:
#Step 3 join dfs
joined_descriptive_stats= suffix_and_join_dfs( 
    imputed_train_descriptive_stats, imputed_predict_descriptive_stats, 'column_name' )

In [10]:
# Step 4 delta
delta_df = get_delta_descriptive_stats_df(joined_descriptive_stats, '2' )

In [11]:
delta_df.show()

+-----------+---------+---------+----------+------------+------------+
|column_name|delta_min|delta_max|delta_mean|delta_stddev|delta_median|
+-----------+---------+---------+----------+------------+------------+
|        ctu|        0|        0|       0.0|         0.0|         0.0|
|   party_id|        0|        0|       0.0|         0.0|         0.0|
|  te_2month|        0|        0|       0.0|         0.0|         0.0|
+-----------+---------+---------+----------+------------+------------+



In [12]:
# Step 5 ks stats
def get_df_with_ks_stats( imputed_train, imputed_predict ):
    columns = imputed_train.schema.names
    col_ks = []
    for col in columns:
        imputed_train_col = imputed_train.select(col).toPandas()[col].tolist()
        imputed_predict_col = imputed_predict.select(col).toPandas()[col].tolist()
        try:
            ks = stats.ks_2samp(imputed_train_col, imputed_predict_col)
            p_value = str(round(ks[0], 2))
            
            kd = str(round(ks[1], 2))

        except Exception as e:
            print('col ',col ,e)
            p_value = ''
            kd = ''      
        col_ks.append((col,p_value, kd))   
    ks_stats_df = spark.createDataFrame(col_ks, ['column_name_ks', 'p_value', 'kd'])
    return ks_stats_df
ks_stats_df = get_df_with_ks_stats (imputed_train, imputed_predict )
ks_stats_df.show()

+--------------+-------+---+
|column_name_ks|p_value| kd|
+--------------+-------+---+
|      party_id|    0.0|1.0|
|           ctu|    0.0|1.0|
|     te_2month|    0.0|1.0|
+--------------+-------+---+



In [14]:
# step 6 KL divergence

In [13]:
# Step 7 Join 
delta_df.join(ks_stats_df, col('column_name') == col('column_name_ks')).show()


+-----------+---------+---------+----------+------------+------------+--------------+-------+---+
|column_name|delta_min|delta_max|delta_mean|delta_stddev|delta_median|column_name_ks|p_value| kd|
+-----------+---------+---------+----------+------------+------------+--------------+-------+---+
|        ctu|        0|        0|       0.0|         0.0|         0.0|           ctu|    0.0|1.0|
|   party_id|        0|        0|       0.0|         0.0|         0.0|      party_id|    0.0|1.0|
|  te_2month|        0|        0|       0.0|         0.0|         0.0|     te_2month|    0.0|1.0|
+-----------+---------+---------+----------+------------+------------+--------------+-------+---+

